# Exploratory Data Analysis
#### by Carlos Carcamo

In this document, I will explore the data sources files for the project, to find any issue on the data. At the end of the analysis I will made a summary with all the cleaning tasks over the data.

In [ ]:
#Import libraries to be used
import pandas as pd
import json

## Profile dataset.csv

Let's start extracting the dataset.csv

In [146]:
#Storing file into Pandas' DataFrame
df = pd.read_csv('dataset.csv', encoding='unicode_escape')

C:\Users\Carcac01\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


> Note: In the extraction phase, I had some troubles with file encoding. There are some characters that are not part of the utf-8 encoding, so I had to change it.


In [186]:
#Read df
df.head()

,ORDER_ID,USER_ID,ORDER_NUMBER,ORDER_DOW,ORDER_HOUR_OF_DAY,ORDER_PRODUCT,ORDER_PROD_AISLE,ORDER_SEQ
0,1000029,54650,4,3,13,Almond Chia Granola Clusters,granola,1
0,1000029,54650,4,3,13,Fresh Cauliflower,fresh vegetables,2
0,1000029,54650,4,3,13,Orange Bell Pepper,fresh vegetables,3
0,1000029,54650,4,3,13,Veggie Chips,chips pretzels,8
0,1000029,54650,4,3,13,Organic Gala Apples,fresh fruits,5


In [187]:
#number of rows and columns
df.shape

(1384617, 8)

In [149]:
#Get datatypes 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131209 entries, 0 to 131208
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   ORDER_ID           131209 non-null  int64 
 1   USER_ID            131209 non-null  int64 
 2   ORDER_NUMBER       131209 non-null  int64 
 3   ORDER_DOW          131209 non-null  object
 4   ORDER_HOUR_OF_DAY  131209 non-null  int64 
 5   ORDER_DETAIL       131209 non-null  object
dtypes: int64(4), object(2)
memory usage: 6.0+ MB


It seems that ORDER_DOW has some invalid values, lets see

In [150]:
df.ORDER_DOW.value_counts()

0          27439
1          19650
6          18878
5          17389
2          16090
4          15941
3          15664
0             26
6             23
3             23
1             22
4             18
Tuesday       16
2             13
5             12
Friday         5
Name: ORDER_DOW, dtype: int64

There are some string values, and some numbers recognized as strings. It will be neccesary to fix it.

Let's explore hours column: 

In [151]:
df.ORDER_HOUR_OF_DAY.value_counts()

14    11103
15    11075
13    10711
12    10622
11    10610
10    10585
16    10495
17     9353
9      9033
18     7558
8      6343
19     5721
20     3978
7      3376
21     3050
22     2442
23     1526
6      1121
0       848
1       507
5       402
2       305
3       223
4       218
24        4
Name: ORDER_HOUR_OF_DAY, dtype: int64

It seems there are 4 orders registered as 24h. As the catalog will be defined from 0 to 23, It will be necessary to fix it.

Regarding to structure of the data source, it seems very obvious some transformations on ORDER_DETAIL column. We have wrapped ther some valius information, like Product Name, Aisle and Order Sequence. Also, we have all the detail from an individualk order wrapped, too. So, let's start unwraping it.

Fisrt step, will be explode the field by Order Detail:

In [152]:
#Split df using ~ chat
df['ORDER_DETAIL_EXPLODED'] = df['ORDER_DETAIL'].str.split(pat='~')
df

,ORDER_ID,USER_ID,ORDER_NUMBER,ORDER_DOW,ORDER_HOUR_OF_DAY,ORDER_DETAIL,ORDER_DETAIL_EXPLODED
0,1000029,54650,4,3,13,Almond Chia Granola Clusters|granola|1~Fresh C...,"[Almond Chia Granola Clusters|granola|1, Fresh..."
1,1003104,124957,10,1,16,Limes|fresh fruits|5~Tomato Juice|juice nectar...,"[Limes|fresh fruits|5, Tomato Juice|juice nect..."
2,1004115,110922,15,0,14,Skim Milk|milk|4~Organic Stage 3 Pumpkin Cranb...,"[Skim Milk|milk|4, Organic Stage 3 Pumpkin Cra..."
3,1004231,12037,7,1,19,Banana|fresh fruits|3~Green Tea Bags|tea|10~Or...,"[Banana|fresh fruits|3, Green Tea Bags|tea|10,..."
4,1004492,196375,16,0,13,Organic Sprouted English Muffin|breakfast bake...,[Organic Sprouted English Muffin|breakfast bak...
...,...,...,...,...,...,...,...
131204,995474,202893,17,2,13,Bag of Organic Bananas|fresh fruits|1~Strawber...,"[Bag of Organic Bananas|fresh fruits|1, Strawb..."
131205,997017,193337,21,5,11,Strawberries|fresh fruits|1~15 Grain Whole Gra...,"[Strawberries|fresh fruits|1, 15 Grain Whole G..."
131206,997622,8661,38,3,9,Antiperspirant Deodorant Cool Essentials Fresh...,[Antiperspirant Deodorant Cool Essentials Fres...
131207,998840,19932,6,2,12,Popcorn|popcorn jerky|1~Bag of Organic Bananas...,"[Popcorn|popcorn jerky|1, Bag of Organic Banan..."


In [153]:
#Explode Dataframe
df = df.explode('ORDER_DETAIL_EXPLODED')
df

,ORDER_ID,USER_ID,ORDER_NUMBER,ORDER_DOW,ORDER_HOUR_OF_DAY,ORDER_DETAIL,ORDER_DETAIL_EXPLODED
0,1000029,54650,4,3,13,Almond Chia Granola Clusters|granola|1~Fresh C...,Almond Chia Granola Clusters|granola|1
0,1000029,54650,4,3,13,Almond Chia Granola Clusters|granola|1~Fresh C...,Fresh Cauliflower|fresh vegetables|2
0,1000029,54650,4,3,13,Almond Chia Granola Clusters|granola|1~Fresh C...,Orange Bell Pepper|fresh vegetables|3
0,1000029,54650,4,3,13,Almond Chia Granola Clusters|granola|1~Fresh C...,Veggie Chips|chips pretzels|8
0,1000029,54650,4,3,13,Almond Chia Granola Clusters|granola|1~Fresh C...,Organic Gala Apples|fresh fruits|5
...,...,...,...,...,...,...,...
131208,999742,121825,10,4,19,Strawberry Preserves|spreads|3~Creamy Peanut B...,Classic Potato Chips|chips pretzels|11
131208,999742,121825,10,4,19,Strawberry Preserves|spreads|3~Creamy Peanut B...,Mozzarella Fresca|packaged cheese|12
131208,999742,121825,10,4,19,Strawberry Preserves|spreads|3~Creamy Peanut B...,Salted Butter Half Sticks|butter|13
131208,999742,121825,10,4,19,Strawberry Preserves|spreads|3~Creamy Peanut B...,"Bathroom Tissue Softness & Strength, Double Ro..."


In [154]:
#Drop ORDER_DETAIL column, as it was exploded
df = df.drop(['ORDER_DETAIL'], axis=1)
df

,ORDER_ID,USER_ID,ORDER_NUMBER,ORDER_DOW,ORDER_HOUR_OF_DAY,ORDER_DETAIL_EXPLODED
0,1000029,54650,4,3,13,Almond Chia Granola Clusters|granola|1
0,1000029,54650,4,3,13,Fresh Cauliflower|fresh vegetables|2
0,1000029,54650,4,3,13,Orange Bell Pepper|fresh vegetables|3
0,1000029,54650,4,3,13,Veggie Chips|chips pretzels|8
0,1000029,54650,4,3,13,Organic Gala Apples|fresh fruits|5
...,...,...,...,...,...,...
131208,999742,121825,10,4,19,Classic Potato Chips|chips pretzels|11
131208,999742,121825,10,4,19,Mozzarella Fresca|packaged cheese|12
131208,999742,121825,10,4,19,Salted Butter Half Sticks|butter|13
131208,999742,121825,10,4,19,"Bathroom Tissue Softness & Strength, Double Ro..."


Now, its time to split the column into 3 new columns:

In [155]:
#Split by | char
df[['ORDER_PRODUCT','ORDER_PROD_AISLE','ORDER_SEQ']] = df['ORDER_DETAIL_EXPLODED'].str.split(pat='|', expand = True)
df

,ORDER_ID,USER_ID,ORDER_NUMBER,ORDER_DOW,ORDER_HOUR_OF_DAY,ORDER_DETAIL_EXPLODED,ORDER_PRODUCT,ORDER_PROD_AISLE,ORDER_SEQ
0,1000029,54650,4,3,13,Almond Chia Granola Clusters|granola|1,Almond Chia Granola Clusters,granola,1
0,1000029,54650,4,3,13,Fresh Cauliflower|fresh vegetables|2,Fresh Cauliflower,fresh vegetables,2
0,1000029,54650,4,3,13,Orange Bell Pepper|fresh vegetables|3,Orange Bell Pepper,fresh vegetables,3
0,1000029,54650,4,3,13,Veggie Chips|chips pretzels|8,Veggie Chips,chips pretzels,8
0,1000029,54650,4,3,13,Organic Gala Apples|fresh fruits|5,Organic Gala Apples,fresh fruits,5
...,...,...,...,...,...,...,...,...,...
131208,999742,121825,10,4,19,Classic Potato Chips|chips pretzels|11,Classic Potato Chips,chips pretzels,11
131208,999742,121825,10,4,19,Mozzarella Fresca|packaged cheese|12,Mozzarella Fresca,packaged cheese,12
131208,999742,121825,10,4,19,Salted Butter Half Sticks|butter|13,Salted Butter Half Sticks,butter,13
131208,999742,121825,10,4,19,"Bathroom Tissue Softness & Strength, Double Ro...","Bathroom Tissue Softness & Strength, Double Rolls",paper goods,4


In [156]:
df = df.drop(['ORDER_DETAIL_EXPLODED'], axis=1)
df

,ORDER_ID,USER_ID,ORDER_NUMBER,ORDER_DOW,ORDER_HOUR_OF_DAY,ORDER_PRODUCT,ORDER_PROD_AISLE,ORDER_SEQ
0,1000029,54650,4,3,13,Almond Chia Granola Clusters,granola,1
0,1000029,54650,4,3,13,Fresh Cauliflower,fresh vegetables,2
0,1000029,54650,4,3,13,Orange Bell Pepper,fresh vegetables,3
0,1000029,54650,4,3,13,Veggie Chips,chips pretzels,8
0,1000029,54650,4,3,13,Organic Gala Apples,fresh fruits,5
...,...,...,...,...,...,...,...,...
131208,999742,121825,10,4,19,Classic Potato Chips,chips pretzels,11
131208,999742,121825,10,4,19,Mozzarella Fresca,packaged cheese,12
131208,999742,121825,10,4,19,Salted Butter Half Sticks,butter,13
131208,999742,121825,10,4,19,"Bathroom Tissue Softness & Strength, Double Rolls",paper goods,4


In [157]:
df

,ORDER_ID,USER_ID,ORDER_NUMBER,ORDER_DOW,ORDER_HOUR_OF_DAY,ORDER_PRODUCT,ORDER_PROD_AISLE,ORDER_SEQ
0,1000029,54650,4,3,13,Almond Chia Granola Clusters,granola,1
0,1000029,54650,4,3,13,Fresh Cauliflower,fresh vegetables,2
0,1000029,54650,4,3,13,Orange Bell Pepper,fresh vegetables,3
0,1000029,54650,4,3,13,Veggie Chips,chips pretzels,8
0,1000029,54650,4,3,13,Organic Gala Apples,fresh fruits,5
...,...,...,...,...,...,...,...,...
131208,999742,121825,10,4,19,Classic Potato Chips,chips pretzels,11
131208,999742,121825,10,4,19,Mozzarella Fresca,packaged cheese,12
131208,999742,121825,10,4,19,Salted Butter Half Sticks,butter,13
131208,999742,121825,10,4,19,"Bathroom Tissue Softness & Strength, Double Rolls",paper goods,4


In [158]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1384617 entries, 0 to 131208
Data columns (total 8 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   ORDER_ID           1384617 non-null  int64 
 1   USER_ID            1384617 non-null  int64 
 2   ORDER_NUMBER       1384617 non-null  int64 
 3   ORDER_DOW          1384617 non-null  object
 4   ORDER_HOUR_OF_DAY  1384617 non-null  int64 
 5   ORDER_PRODUCT      1384617 non-null  object
 6   ORDER_PROD_AISLE   1384617 non-null  object
 7   ORDER_SEQ          1384617 non-null  object
dtypes: int64(4), object(4)
memory usage: 95.1+ MB


Let's continue grouping the order by ORDER_ID, just to find if is there any outlier number of products for an specific order:

In [159]:
grouped_df = df.groupby(['ORDER_ID'])['ORDER_ID'].agg([len])
grouped_df.sort_values(by='len', ascending=False)

,len
ORDER_ID,
1395075,80
2813632,80
949182,77
2869702,76
341238,76
...,...
1144944,1
1144765,1
1144608,1


An order with 80 products can be considered as normal, but let's explore that particular case:

In [160]:
df[df['ORDER_ID'] == 1395075].sort_values(by='ORDER_SEQ')

,ORDER_ID,USER_ID,ORDER_NUMBER,ORDER_DOW,ORDER_HOUR_OF_DAY,ORDER_PRODUCT,ORDER_PROD_AISLE,ORDER_SEQ
66256,1395075,197541,13,1,14,Original Liquid Laundry Detergent,laundry,1
66256,1395075,197541,13,1,14,Hemorrhoidal Ointment,digestion,10
66256,1395075,197541,13,1,14,Double Zipper Gallon Freezer Bags,food storage,11
66256,1395075,197541,13,1,14,Quart Storage Bags,food storage,12
66256,1395075,197541,13,1,14,Reduced Fat Creamy Peanut Butter Spread,spreads,13
...,...,...,...,...,...,...,...,...
66256,1395075,197541,13,1,14,Organic Baby Bella Mushrooms,packaged vegetables fruits,78
66256,1395075,197541,13,1,14,Organic Avocado,fresh fruits,79
66256,1395075,197541,13,1,14,Shower Tub & Tile Cleaner with Oxy Clean,cleaning products,8
66256,1395075,197541,13,1,14,Mixed Nuts Lightly Salted,nuts seeds dried fruit,80


Find duplicated rows:

In [161]:
df[df.duplicated()]

,ORDER_ID,USER_ID,ORDER_NUMBER,ORDER_DOW,ORDER_HOUR_OF_DAY,ORDER_PRODUCT,ORDER_PROD_AISLE,ORDER_SEQ


No duplicated rows were found.

now, let's check if the corelation between ORDER_ID and ORder_NUM is the same


In [179]:
grouped_df = df.groupby(['ORDER_ID','ORDER_NUMBER'])['ORDER_ID'].agg([len])
##grouped_df.sort_values(by='len', ascending=False)
grouped_df

,,len
ORDER_ID,ORDER_NUMBER,
1,4,8
36,23,8
38,6,9
96,7,7
98,41,49
...,...,...
3421049,4,6
3421056,13,5
3421058,20,8


## Profile product.json file

Now, its time to check the json file:

In [162]:
with open('products.json','r',encoding='utf-8') as json_file:
    data =json.load(json_file)

    #Navigate into json file, to find the items element
list_results = data["results"]
dict_products = list_results[0]
items_list = dict_products["items"]

#Load json into Pandas' Dataframe
df_json = pd.DataFrame(items_list, columns = ['product_name','aisle','department'])

df_json

,product_name,aisle,department
0,Chocolate Sandwich Cookies,cookies cakes,snacks
1,All-Seasons Salt,spices seasonings,pantry
2,Robust Golden Unsweetened Oolong Tea,tea,beverages
3,Smart Ones Classic Favorites Mini Rigatoni Wit...,frozen meals,frozen
4,Green Chile Anytime Sauce,marinades meat preparation,pantry
...,...,...,...
49683,"Vodka, Triple Distilled, Twist of Vanilla",spirits,alcohol
49684,En Croute Roast Hazelnut Cranberry,frozen vegan vegetarian,frozen
49685,Artisan Baguette,bread,bakery
49686,Smartblend Healthy Metabolism Dry Cat Food,cat food care,pets


In [163]:
df_json['aisle'].unique()

array(['cookies cakes', 'spices seasonings', 'tea', 'frozen meals',
       'marinades meat preparation', 'cold flu allergy', 'juice nectars',
       'frozen produce', 'yogurt', 'water seltzer sparkling water',
       'refrigerated', 'frozen dessert', 'dish detergents',
       'diapers wipes', 'ice cream toppings', 'poultry counter',
       'frozen pizza', 'grains rice dried goods', 'dog food care',
       'oral hygiene', 'packaged poultry', 'vitamins supplements',
       'energy granola bars', 'cat food care', 'body lotions soap',
       'cereal', 'canned jarred vegetables', 'packaged vegetables fruits',
       'chips pretzels', 'dry pasta', 'hot dogs bacon sausage',
       'soup broth bouillon', 'missing', 'energy sports drinks',
       'crackers', 'fresh vegetables', 'frozen breads doughs',
       'asian foods', 'more household', 'tofu meat alternatives',
       'doughs gelatins bake mixes', 'hair care',
       'specialty wines champagnes', 'popcorn jerky',
       'plates bowls cups 

In [164]:
grouped_df = df_json.groupby(['aisle'])['aisle'].agg([len])
grouped_df.sort_values(by='len', ascending=False)

,len
aisle,
missing,1258
candy chocolate,1246
ice cream ice,1091
vitamins supplements,1038
yogurt,1026
...,...
frozen juice,47
baby accessories,44
packaged produce,32


It seems there are a lot of missing values for aisle. Those values must be sent to the client/business for checks and to correct the data. For this moment, I will treat them as correct data, as I can't define the correct aisle, and also, I can't just erase the data as I will be losig a significant part of the products.

Let's check the department field:

In [165]:
grouped_df = df_json.groupby(['department'])['department'].agg([len])
grouped_df.sort_values(by='len', ascending=False)

,len
department,
personal care,6563
snacks,6264
pantry,5371
beverages,4365
frozen,4007
dairy eggs,3449
household,3085
canned goods,2092
dry goods pasta,1858


Is a similar issue. I will continue treat them as correct values.

Now, I will check for duplicity:

In [185]:
df_json[df_json.duplicated()]

,product_name,aisle,department


There are no duplicates on json file.

In [166]:
df_json[df_json.product_name.str.contains("Microwavable Cut Resistant Plates")]

,product_name,aisle,department
28030,"8.5\"" Microwavable Cut Resistant Plates",plates bowls cups flatware,household


In [167]:
df_json['product_name'] = df_json['product_name'].str.replace('\\','')

In [168]:
df_json[df_json.product_name.str.contains("Organic Chocolate Cake")]

,product_name,aisle,department
9928,"6"" Organic Chocolate Cake",bakery desserts,bakery
13799,Organic Chocolate Cakes,frozen dessert,frozen
22416,Organic Chocolate Cake Mix,doughs gelatins bake mixes,pantry


In [169]:
inner_join_df = pd.merge(df,df_json,left_on='ORDER_PRODUCT', right_on='product_name')
inner_join_df

,ORDER_ID,USER_ID,ORDER_NUMBER,ORDER_DOW,ORDER_HOUR_OF_DAY,ORDER_PRODUCT,ORDER_PROD_AISLE,ORDER_SEQ,product_name,aisle,department
0,1000029,54650,4,3,13,Almond Chia Granola Clusters,granola,1,Almond Chia Granola Clusters,granola,breakfast
1,605562,50291,10,2,15,Almond Chia Granola Clusters,granola,13,Almond Chia Granola Clusters,granola,breakfast
2,1314091,152359,21,0,16,Almond Chia Granola Clusters,granola,9,Almond Chia Granola Clusters,granola,breakfast
3,1462113,55876,9,1,14,Almond Chia Granola Clusters,granola,4,Almond Chia Granola Clusters,granola,breakfast
4,83463,177131,15,2,16,Almond Chia Granola Clusters,granola,17,Almond Chia Granola Clusters,granola,breakfast
...,...,...,...,...,...,...,...,...,...,...,...
1383309,960112,89556,5,3,11,Active Blueberry Pomegranate Clear Nutritional...,protein meal replacements,1,Active Blueberry Pomegranate Clear Nutritional...,protein meal replacements,personal care
1383310,963719,117430,4,6,16,Chocolate Chip Heart Thrive,bakery desserts,5,Chocolate Chip Heart Thrive,bakery desserts,bakery
1383311,963719,117430,4,6,16,Skinner's Raisin Bran Cereal,cereal,10,Skinner's Raisin Bran Cereal,cereal,breakfast
1383312,968144,175649,5,6,18,Rainbow Carrots,fresh vegetables,10,Rainbow Carrots,fresh vegetables,produce


In [170]:
df.shape[0]-inner_join_df.shape[0]

1303

In [184]:
inner_join_df[inner_join_df['ORDER_PROD_AISLE']!=inner_join_df['aisle']]

,ORDER_ID,USER_ID,ORDER_NUMBER,ORDER_DOW,ORDER_HOUR_OF_DAY,ORDER_PRODUCT,ORDER_PROD_AISLE,ORDER_SEQ,product_name,aisle,department


In [171]:
outer_join_df = pd.merge(df,df_json,left_on='ORDER_PRODUCT', right_on='product_name', how='outer',indicator=True)
outer_join_df = outer_join_df[outer_join_df['product_name'].isnull()]
outer_join_df

,ORDER_ID,USER_ID,ORDER_NUMBER,ORDER_DOW,ORDER_HOUR_OF_DAY,ORDER_PRODUCT,ORDER_PROD_AISLE,ORDER_SEQ,product_name,aisle,department,_merge
839446,1440136.0,81449.0,6.0,6,18.0,Flip Low-Fat Almond Coco Loco Lowfat Greek Yo...,yogurt,6,NaN,NaN,NaN,left_only
839447,1683797.0,192346.0,6.0,5,12.0,Flip Low-Fat Almond Coco Loco Lowfat Greek Yo...,yogurt,28,NaN,NaN,NaN,left_only
839448,2288931.0,76181.0,7.0,5,15.0,Flip Low-Fat Almond Coco Loco Lowfat Greek Yo...,yogurt,10,NaN,NaN,NaN,left_only
839449,36580.0,112691.0,41.0,0,14.0,Flip Low-Fat Almond Coco Loco Lowfat Greek Yo...,yogurt,15,NaN,NaN,NaN,left_only
839450,420767.0,112528.0,6.0,4,16.0,Flip Low-Fat Almond Coco Loco Lowfat Greek Yo...,yogurt,2,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...
1381236,2710094.0,116856.0,5.0,0,19.0,Infusions Ultra Downy® Infusions Lavender Ser...,laundry,7,NaN,NaN,NaN,left_only
1382422,250181.0,162884.0,5.0,6,12.0,Kickin BBQ Chopped Salad,packaged vegetables fruits,9,NaN,NaN,NaN,left_only
1382423,695462.0,119586.0,7.0,6,15.0,Kickin BBQ Chopped Salad,packaged vegetables fruits,4,NaN,NaN,NaN,left_only
1382424,218685.0,46783.0,8.0,6,18.0,Kickin BBQ Chopped Salad,packaged vegetables fruits,2,NaN,NaN,NaN,left_only


In [172]:
df_json[df_json.product_name.str.contains("Low-Fat Almond")]

,product_name,aisle,department
32330,Flip™ Low-Fat Almond Coco Loco Lowfat Greek Yo...,yogurt,dairy eggs


In [174]:
outer_join_df = pd.merge(df,df_json,left_on='ORDER_PRODUCT', right_on='product_name', how='outer',indicator=True)
outer_join_df = outer_join_df[outer_join_df['ORDER_PRODUCT'].isnull()]
outer_join_df

,ORDER_ID,USER_ID,ORDER_NUMBER,ORDER_DOW,ORDER_HOUR_OF_DAY,ORDER_PRODUCT,ORDER_PROD_AISLE,ORDER_SEQ,product_name,aisle,department,_merge
1384617,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dry Nose Oil,cold flu allergy,personal care,right_only
1384618,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rendered Duck Fat,poultry counter,meat seafood,right_only
1384619,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gluten Free Quinoa Three Cheese & Mushroom Blend,grains rice dried goods,dry goods pasta,right_only
1384620,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pomegranate Cranberry & Aloe Vera Enrich Drink,juice nectars,beverages,right_only
1384621,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tri-Vi-Sol® Vitamins A-C-and D Supplement Drop...,vitamins supplements,personal care,right_only
...,...,...,...,...,...,...,...,...,...,...,...,...
1395231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ginger Snaps Snacking Cookies,cookies cakes,snacks,right_only
1395232,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,K Cup Dark Blend,missing,missing,right_only
1395233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stone Baked Multi Grain Artisan Rolls,frozen appetizers sides,frozen,right_only
1395234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Vodka, Triple Distilled, Twist of Vanilla",spirits,alcohol,right_only


In [176]:
outer_join_df = pd.merge(df,df_json,left_on='ORDER_PRODUCT', right_on='product_name', how='outer',indicator=True)
outer_join_df = outer_join_df[outer_join_df['ORDER_PRODUCT'].isnull()]
outer_join_df[outer_join_df['aisle']=='missing']

,ORDER_ID,USER_ID,ORDER_NUMBER,ORDER_DOW,ORDER_HOUR_OF_DAY,ORDER_PRODUCT,ORDER_PROD_AISLE,ORDER_SEQ,product_name,aisle,department,_merge
1384686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Classics Baby Binks Easter Chocolate Bunny,missing,missing,right_only
1384691,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Strawberry Cheesecake Nonfat Yogurt,missing,missing,right_only
1384693,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Crossovers Olive Thyme Almond,missing,missing,right_only
1384719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bake & Break Cheese Garlic Loaf,missing,missing,right_only
1384722,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Traditional Tartar Sauce,missing,missing,right_only
...,...,...,...,...,...,...,...,...,...,...,...,...
1395171,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cinnamon Coffee Cake,missing,missing,right_only
1395185,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Black Tea With Vanilla Flavor In Pyramid Bags,missing,missing,right_only
1395197,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Custard Caramel Sea Salt,missing,missing,right_only
1395206,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chocolate Vanilla Malt Custard,missing,missing,right_only


## Changes proposed for dataset.csv

#### Data changes
- Replace string values on ``ORDER_DOW`` column
- Replace "24" value for "0" on ``ORDER_HOUR_OF_DAY`` column
- Take in count encoding for special characters
- In the extraction phase,I have encountered some products with "?" character. I had to replace it to continue with the join with products

#### Data Structure changes
- Explode ``ORDER_DETAIL`` column by '~' character
- Split exploded ``ORDER_DETAIL`` column by '|' character

## Changes proposed for product.json

#### Data changes
- Remove '\\' from product_name column
- Take in count "missing" value for ``aisle`` and ``department`` columns
- Take in count encoding for special characters

#### Data Structure changes
- Take in count there is no unique key for the structure. Joins with order must be done on ``product_name`` column
